# Predict VoxCeleb utterance info from SSL representations

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [2]:
from notebooks.notebooks_utils import (
    load_models,
    evaluate_models,
    create_metrics_df
)

from sslsv.evaluation.CosineSVEvaluation import CosineSVEvaluation, CosineSVEvaluationTaskConfig

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
torchvision is not available - cannot save figures


In [3]:
models = load_models([
    'configs/simclr_ecapa.yml',
    'configs/simclr_ecapa_simo.yml'
])

In [4]:
evaluate_models(models, CosineSVEvaluation, CosineSVEvaluationTaskConfig())

Extracting test embeddings: 100%|█████████████████████████████████████████████████████████| 74/74 [00:14<00:00,  4.96it/s]


In [9]:
from sklearn.linear_model import LogisticRegression

def fit_mlp_on_representations(model, y_key_pos):
    keys = list(model['embeddings'].keys())
    
    X = [model['embeddings'][key][0].numpy() for key in keys]
    if y_key_pos is None:
        y = keys
    else:
        y = [key.split('/')[y_key_pos] for key in keys]
    
    clf = LogisticRegression()
    clf.fit(X, y)
    
    print(f'Accuracy: {clf.score(X, y)}')
    
    return clf, X, y

## Speaker ID

In [11]:
_ = fit_mlp_on_representations(models['simclr_ecapa'], y_key_pos=1)

Accuracy: 0.9978759558198811


In [12]:
_ = fit_mlp_on_representations(models['simclr_ecapa_simo'], y_key_pos=1)

Accuracy: 0.995964316057774


## Video ID

In [13]:
_ = fit_mlp_on_representations(models['simclr_ecapa'], y_key_pos=2)

Accuracy: 0.524214103653356


In [14]:
_ = fit_mlp_on_representations(models['simclr_ecapa_simo'], y_key_pos=2)

Accuracy: 0.5259133389974512


## Segment ID

In [15]:
_ = fit_mlp_on_representations(models['simclr_ecapa'], y_key_pos=3)

Accuracy: 0.18118096856414614


/lre/home/tlepage/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
_ = fit_mlp_on_representations(models['simclr_ecapa_simo'], y_key_pos=3)

Accuracy: 0.16652506372132542


/lre/home/tlepage/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Sample ID

In [18]:
_ = fit_mlp_on_representations(models['simclr_ecapa'], y_key_pos=None)

Accuracy: 0.9972387425658453
